In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [0]:
!pip install mlflow

In [0]:
import mlflow

### Model: Random Forest

In [0]:
X_train = pd.read_csv("/dbfs/FileStore/shared_uploads/agusbaffo@gmail.com/X_train_prpro.csv")
y_train = pd.read_csv("/dbfs/FileStore/shared_uploads/agusbaffo@gmail.com/y_train_prpro.csv")
X_test = pd.read_csv("/dbfs/FileStore/shared_uploads/agusbaffo@gmail.com/X_test_prpro.csv")
y_test = pd.read_csv("/dbfs/FileStore/shared_uploads/agusbaffo@gmail.com/y_test_prpro.csv")

In [0]:
from sklearn.model_selection import train_test_split

# Split into train/cv
X_train_rf, X_cv_rf, y_train_rf, y_cv_rf = train_test_split(X_train, y_train,
                                                            test_size=0.2,
                                                            stratify=y_train)

In [0]:
def testModel(clf_,grid_values_,X_train_,y_train_,X_cv_,y_cv_,scoring_='f1',verbose_=0):
    # model.get_params() # Return selected params
    # cv=None -> None, to use the default 5-fold cross validation (K-Fold, k=5)
    model_ = GridSearchCV(clf_, param_grid = grid_values_, cv=None, scoring=scoring_,verbose=verbose_)
    model_.fit(X_train_, y_train_)

    y_pred = model_.predict(X_cv_)
    metrics_={    
        "accuracy":         metrics.accuracy_score(y_cv_, y_pred),
        "precision":        metrics.precision_score(y_cv_, y_pred),
        "recall":           metrics.recall_score(y_cv_, y_pred),
        "f1_score":         metrics.f1_score(y_cv_, y_pred),
        "confusion_matrix": metrics.confusion_matrix(y_cv_, y_pred)
    }
    return model_, metrics_

In [0]:
mlflow.start_run(experiment_id = '1224067848736773')

Out[11]: <ActiveRun: >

In [0]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(criterion="entropy")
grid_values = {}
model, model_metrics = testModel(clf,grid_values,X_train_rf,y_train_rf,X_cv_rf,y_cv_rf)

/databricks/python/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
 estimator.fit(X_train, y_train, **fit_params)
/databricks/python/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
 estimator.fit(X_train, y_train, **fit_params)
/databricks/python/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
 estimator.fit(X_train, y_train, **fit_params)
/databricks/python/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
 estimator.fit(X_train, y_train, **fit_params)
/databricks/python/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
 estimator.fit(X_train, y_train, **fit_params)
/databricks/python/lib/python3.7/site-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
 self.best_estimator_.fit(X, y, **fit_params)

In [0]:
model_metrics

Out[13]: {'accuracy': 0.9570925237704523,
 'precision': 0.968931138418734,
 'recall': 0.9308386234547277,
 'f1_score': 0.9495029821073558,
 'confusion_matrix': array([[11472, 268],
 [ 621, 8358]])}

In [0]:
mlflow.sklearn.log_model(model,"model")
mlflow.set_tag("Owner","abaffo")
mlflow.set_tag("Model Description","RandomForest")
mlflow.log_metric("accuracy",model_metrics['accuracy'])
mlflow.log_metric("precision",model_metrics['precision'])
mlflow.log_metric("recall",model_metrics['recall'])
mlflow.log_metric("f1_score",model_metrics['f1_score'])

In [0]:
mlflow.end_run()